<div id="top"></div>

# Project: Investigate a Dataset - [Amazon_product_data]

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#question">Questions</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<li><a href="#limit">Limitation</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 


#### Dataset Overview
<ul>
<li>This dataset contains detailed information about 1,465 Amazon products.</li>
<li>The dataset includes various attributes related to product pricing, discounts, ratings, reviews, and product categories.</li>
<li>Column named category contains multiple values separated by a pipe (|). This represent hierarchical category assignments for a single product.</li>
<li>Dataset contains minimal missing values.</li>
</ul>

#### Columns in the Dataset:
1. `product_id`: A unique identifier for each product.
2. `product_name`: Name of the Product.
3. `category`: Category of the Product.
4. `discounted_price`: Discounted Price of the Product.
5. `actual_price`: Actual Price of the Product.
6. `discount_percentage`: Percentage of Discount for the Product.
7. `rating`: Rating of the Product.
8. `rating_count`: Number of people who voted for the Amazon rating.
9. `about_product`: Description about the Product.
10. `user_id`: ID of the user who wrote review for the Product.
11. `user_name`: Name of the user who wrote review for the Product.
12. `review_id`: ID of the user review.
13. `review_title`: Short review.
14. `review_content`: Long review.
15. `img_link`: Image Link of the Product.
16. `product_link`: Official Website Link of the Product

<a id='question'></a>
## Question(s) for Analysis
<ol>
    <li><a href="#question1">Are higher discount percentages are associated with lower product ratings?</a>
        <ul>
            <li>This question determine whether products offerd at higher discounts tend to have lower rating.</li>
            <li>It leads to shows that discounted items maybe perceived as lower quality or less desirable.</li>
            <li>This relationship will identify optimal discount levels that do not negatively impact product perception.</li>
        </ul>
    </li>
    <li><a href="#question2">Do products with more reviews have higher average ratings?</a>
        <ul>
            <li>This hypothesis investigates if products with higher number of reviews tend to have higher rating.</li>
            <li>It determine if popular items receive better feedback.</li>
        </ul>
    </li>
    <li><a href="#question3">Do higher-priced products receive better ratings than lower-priced ones?</a>
        <ul>
            <li>Consumers might have higher expectations for premium products, so it’s interesting to see if higher-priced products actually receive better ratings. Testing this hypothesis can reveal if price influences customer perception positively or if lower-priced products perform equally well in terms of customer satisfaction.</li>
        </ul>
    </li>
    <li><a href="#question4">Do Products with Positive Sentiment in Reviews Have Higher Ratings?</a>
        <ul>
            <li>This hypothesis examines whether the sentiment expressed in customer reviews (positive, neutral, or negative) aligns with the product’s rating. If confirmed, this could suggest that sentiment analysis can serve as a proxy for predicting overall product satisfaction and help brands understand customer sentiment trends.</li>
        </ul>
    </li>
    <li><a href="#question5">Which Categories of Products Receive the Highest or Lowest Ratings?</a>
        <ul>
            <li>Different product categories (like Electronics, Home & Kitchen) may naturally attract different rating distributions due to factors such as user expectations and product complexity. This hypothesis will help identify if certain categories are more favorably rated, providing insights into customer satisfaction by category.</li>
        </ul>
    </li>
</ol>

In [1]:
# import numpy, pandas, matplotlib, and seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
from scipy import stats
import plotly.io as pio
pio.renderers.default='notebook'
%matplotlib inline
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Upgrade pandas to use dataframe.explode() function. 
#!pip install --upgrade pandas==0.25.1

<a id='wrangling'></a>
## Data Wrangling

### Initial Cleaning

In [3]:
# load datasets from amazon.csv file
df = pd.read_csv("data/amazon.csv")

In [4]:
# view the first 5 rows of the dataset
df.head()

In [5]:
#view dimensions of dataset
df.shape

The number of samples in the dataset is 1465, The number of columns in the dataset is 16

In [6]:
#explore the dataset, checking for the missing value of each column.
df.info()

There is no null data in this dataset.

In [7]:
#view the datatype of each column
df.dtypes

In [8]:
#Show how many unique values are there in each columns.
df.nunique()

In [9]:
#show the null rows in each column
df.isnull().sum()

There are 2 null rows in rating_count column

In [10]:
#view the overall information of the dataset:
df.describe()

In [11]:
df.rating.value_counts()

`Rating` column has 1 row that contain '|' symbol

In [12]:
#Create a copy of data to clean named df_clean
df_clean = df.copy()

### Fixing `rating` Column

In [13]:
# General look in rows contain '|' symbol
df[df['rating'] =='|']

I checked the product_linke of "B08L12N5H1", the rating is 3.9, so I will fill the rating at 3.9

In [14]:
df_clean.loc[df_clean['product_id'] == 'B08L12N5H1', 'rating'] = '3.9'

### Drop extraneous columns
Columns to Drop: `img_link` , `user_name` , `user_id`

In [15]:
#drop columns from the dataset
df_clean.drop(['img_link','user_name','user_id'], inplace=True, axis=1)

In [16]:
#confirm the changes
df_clean.info()

### Dedupe Data

In [17]:
#print number of duplicates to confirm dedupe - should be 0
print(df_clean.duplicated().sum())

No duplicate information, so it doesn't need any further cleaning.

### Fix Rows with Missing Values

In [18]:
# Define which row in rating_count is null
df_clean[df_clean['rating_count'].isnull()]

I go to product link to check and notice that product_id: B0B94JPY2N has 4.0 rating and 58 rating_count. 
Product_id B0BQRJ3C47 hasn't existed in amazon so I will drop it.

In [19]:
# Change row with product_id = 'B0B94JPY2N': rating to 4.0 and rating_count to 58
df_clean.loc[df_clean['product_id'] == 'B0B94JPY2N', 'rating_count'] = '58'
df_clean.loc[df_clean['product_id'] == 'B0B94JPY2N', 'rating'] = '4.0'

In [20]:
# Then drop rows with any null values in dataset which is Product_id B0BQRJ3C47
df_clean.dropna(inplace=True)

In [21]:
#show the null rows in each column - should 0 for all columns
df_clean.isnull().sum()

### Data Transformation

Create a float_change function to automatically change float type to each column.

In [22]:
def convert_to_float(df,column_name):
    # removes any non-numeric characters (except for the decimal point)
    df[column_name] = df[column_name].str.replace(r'[^0-9.]', '', regex=True)
    
    # convert string to float for actual_price column
    df[column_name] = df[column_name].astype(float)

Convert price-related columns to floats for easier calculations

In [23]:
# convert string to float for discounted_price column
convert_to_float(df_clean,'discounted_price')

In [24]:
# convert string to float for actual_price column
convert_to_float(df_clean,'actual_price')

In [25]:
# convert string to float for rating_count column
convert_to_float(df_clean,"rating_count")

In [26]:
# convert string to float for rating column
convert_to_float(df_clean,"rating")

In [27]:
# Handle the 'discount_percentage' column by converting it to a decimal value (0–1 range)
df_clean['discount_percentage'] = df_clean['discount_percentage'].str.replace('%', '').astype(float) / 100

In [28]:
# check data type for revenue column and budget column 
# discounted_price, actual_price, rating_count, rating, discount_percentage should be float
df_clean.dtypes

In [29]:
df_clean.isnull().sum()

### Feature Creating

Create a split_and_expand function to automatically split and expand into separate rows.

In [30]:
# Split the 'category' column into two parts: general category and product type
def split_category(df, column_name, fill_value='NaN'):
    """
    Splits a specified column on the '|' delimiter and expands it into two separate columns.
    
    Parameters:
    df (DataFrame): The DataFrame containing the column to split.
    column_name (str): The name of the column to split.
    
    Returns:
    DataFrame: The modified DataFrame with split and expanded columns.
    """
    
    # split the specified column on the delimiter '|'
    # expand=True: instead of storing lists in a single column,
    # it creates multiple columns in the resulting DataFrame category_split
    
    split_column = df[column_name].str.split('|', expand=True) 
    
    # Create new column named main_category
    df['main_category'] = split_column[0]
    
    # Create new column named product_type
    df['product_type'] = split_column[1]
    
    #drop the old columns
    df.drop(columns=[column_name], inplace=True)

Call split_category function to split category column

In [31]:
# Appply the function to 'category' column
split_category(df_clean, 'category')

In [32]:
# Using the median price to split the products
median_price = df_clean['actual_price'].median()
df_clean['price_category'] = df_clean['actual_price'].apply(lambda x: 'Lower-Priced' if x < median_price else 'Higher-Priced')

Function to get sentiment

In [33]:
# Initialize VADER SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [34]:
# Function to get sentiment
def get_vader_sentiment(text):
    score = sia.polarity_scores(text)
    return score['compound']  # compound score between -1 (negative) and 1 (positive)

In [35]:
# Applying sentiment analysis
df_clean['vader_sentiment_score'] = df_clean['review_content'].apply(get_vader_sentiment)
df_clean['vader_sentiment'] = df_clean['vader_sentiment_score'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

In [36]:
df_clean['vader_sentiment'].value_counts()

Grouping Rating

**Reasoning**: This grouping gives a meaningful breakdown, because the ratings are mostly concentrated around certain values.

- Low Ratings (1.0 - 3.0): These are rare, but they represent a significant drop in product quality or satisfaction.
- Average Ratings (3.0 - 4.0): This range covers a moderate level of satisfaction.
- Good Ratings (4.0 - 4.5): The largest concentration falls here, representing well-rated products.
- Excellent Ratings (4.5 - 5.0): This would represent top-rated products.

In [37]:
df_clean['mean_of_ratings'] = pd.cut(df_clean['rating'], [1, 3, 3.5, 4, 5], labels = ['Low', 'Average', 'Good', 'Excellent'])

Binning rating_count into defined groups.

**Reasoning**: This will help reduce the impact of extreme values.

In [38]:
# Define bins for review count ranges
bins = [0, 100, 1000, 5000, 10000, 50000, 100000, 500000]  # Define the review count ranges
labels = ['0-100', '100-1000', '1000-5000', '5000-10000', '10000-50000', '50000-100000', '100000+']  # Label each range

In [39]:
# Create a new column in the DataFrame to represent review count ranges
df_clean['rating_count_group'] = pd.cut(df_clean['rating_count'], bins=bins, labels=labels)

In [40]:
#Confirm the new columns
df_clean.info()

<a id='eda'></a>
## Exploratory Data Analysis

<a id='question1'></a>
### Research Question 1: Is there a relationship between discount percentage and average product rating ?

**Analysis Decision 1**: Bin the discount percentage data

In [41]:
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
df_clean['discount_bin'] = pd.cut(df_clean['discount_percentage'], bins=bins, labels=labels)

**Reasoning**: Provids a simplified distribution and can make comparisons easier.

**Analysis Decision 2**: Draw histogram to describe distribution of discount percentage.

In [42]:
plt.figure(figsize=(14, 8))
sns.histplot(df_clean['discount_bin'], kde=True)
plt.title('Distribution of Discounted Percentage')
plt.xlabel('Discounted Percentage')
plt.ylabel('Frequency')
plt.show()

**Reasoning**: This histogram plot will help us understand how pricing stategies are distributed and potential impact on consumer behavior and busisness performance.

**Summary**: 
- The peak around 40% to 60% discount.
- Discounts greater than 80% are rare.
- Very few products are offered without any discount.
- Most products are offered with moderate to high discounts, with a few products available at no discount or very high discounts.

**Analysis Decision 3**: Rating

In [43]:
plt.figure(figsize=(14, 8))
sns.histplot(df_clean['rating'], kde=True)
plt.title('Distribution of Rating')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

**Reasoning**: This histogram plot will help us understand how pricing stategies are distributed and potential impact on consumer behavior and busisness performance.

**Summary**: 
- The majority of ratings seem to cluster betwen 3.5 and 4.5, indicating that most products are rated positively.
- Very few ratings are below 3.0, suggesting that poor ratings are rare in this dataset

In [44]:
#Calculate the correlation number between vote_count and Revenue
df_clean['discount_percentage'].corr(df_clean['rating'])

**Summary**: 
- The correlation between discount percentage and rating is -0.155, it shows a weak negative relationship. This implies that in general, when the product rating decreases, the discount percentage will be increases, though the effect is minimal.

**Analysis Decision 4**: Relationship between rating and discount percentage

In [45]:
sns.scatterplot(x='rating',y='discount_percentage', data= df_clean, color = "green")
plt.title("discount_percentage & rating")
plt.show()

**Reasoning**: This scatter plot will help us understand whether higher rating engagement leads to higher discount percentage

**Summary**:
- The scatter plot shows a very slight downward trend, but it is difficult to tell.

Visualize for the distribution of mean of rating.

In [46]:
plt.figure(figsize=(14, 8))
sns.histplot(df_clean['mean_of_ratings'], kde=True)
plt.title('Distribution of Rating')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

The plot shows that the dataset mostly give excellent rating in the product.

**Summary**:
- The scatter plot shows a very slight downward trend, but it is difficult to tell.

In [47]:
discount_vs_rating = df_clean.groupby('mean_of_ratings')['discount_percentage'].mean()
print(discount_vs_rating)

**Summary**:
- Low-rated products have the highest average discount at 63.9%.
- Excellent-rated products have the lowest average discount at 45.4%
- This suggests that lower-rated products tend to have higher discounts.

In [48]:
plt.figure(figsize=(14,8))
sns.boxplot(x='mean_of_ratings', y='discount_percentage', data=df_clean)
plt.title('Box Plot of Discount Percentage Across Rating Categories')
plt.xlabel('Rating Category')
plt.ylabel('Discount Percentage')
plt.show()

The relationship between discount percentage and product ratings is clear:
- Products with lower ratings tend to offer higher discounts.
- Products with higher ratings offer more moderate discounts, leveraging their quality and reputation to drive sales without needing large discounts.

In [49]:
# Group by rating categories and calculate the correlation
grouped_by_rating = df_clean.groupby('mean_of_ratings').apply(
    lambda x: x['discount_percentage'].corr(x['rating'])
)

print(grouped_by_rating)

- Low-Rated Products: There is a moderately strong positive correlation for low-rated products. This suggests that products with low ratings are likely to offer higher discounts.
- Average, Good, and Excellent Products: For these categories, the correlation is weakly negative. This suggests that higher discounts are less commonly associated with higher-rated products. In fact, these products are more likely to maintain moderate or lower discount levels, relying on their good ratings to drive sales.

**Analysis Decision 6**: Analyze A/B Test Results

Split the dataset into 3 groups based on discount ranges: (0-30%), (30-60%), and (60-100%)

In [50]:
# Define groups based on discount ranges
low_discount = df_clean[(df_clean['discount_percentage'] >= 0) & (df_clean['discount_percentage'] <= 0.3)]
medium_discount = df_clean[(df_clean['discount_percentage'] > 0.3) & (df_clean['discount_percentage'] <= 0.6)]
high_discount = df_clean[(df_clean['discount_percentage'] > 0.6) & (df_clean['discount_percentage'] <= 1)]

Null Hypothesis $H_0$ for each groups: 
- There is no significant different in average product ratings between low discounts (0-30%) and medium discount (30-60%)
- There is no significant different in average product ratings between medium discount (30-60%) and high discount (60-100%)
- There is no significant different in average product ratings between low discounts (0-30%) and high discount (60-100%)

In [51]:
# Count successes (ratings >= 4.0) for each group
low_success = low_discount[low_discount['rating'] >= 4.0].shape[0]
medium_success = medium_discount[medium_discount['rating'] >= 4.0].shape[0]
high_success = high_discount[high_discount['rating'] >= 4.0].shape[0]

In [52]:
# Count total observations for each group
low_total = low_discount.shape[0]
medium_total = medium_discount.shape[0]
high_total = high_discount.shape[0]

In [53]:
# Perform proportions Z-test between Low and Medium discounts
success_counts_lm = [low_success, medium_success]
sample_sizes_lm = [low_total, medium_total]
z_stat_lm, p_value_lm = proportions_ztest(success_counts_lm, sample_sizes_lm)

In [54]:
# Perform proportions Z-test between Medium and High discounts
success_counts_mh = [medium_success, high_success]
sample_sizes_mh = [medium_total, high_total]
z_stat_mh, p_value_mh = proportions_ztest(success_counts_mh, sample_sizes_mh)

In [55]:
# Perform proportions Z-test between Low and High discounts
success_counts_lh = [low_success, high_success]
sample_sizes_lh = [low_total, high_total]
z_stat_lh, p_value_lh = proportions_ztest(success_counts_lh, sample_sizes_lh)

In [56]:
# Summarize results
results = {
    "Low Discounts (0-30%) Total": low_total,
    "Low Discounts Successes": low_success,
    "Medium Discounts (30-60%) Total": medium_total,
    "Medium Discounts Successes": medium_success,
    "High Discounts (60-100%) Total": high_total,
    "High Discounts Successes": high_success,
    "Low vs Medium Z-Statistic": z_stat_lm,
    "Low vs Medium P-Value": p_value_lm,
    "Medium vs High Z-Statistic": z_stat_mh,
    "Medium vs High P-Value": p_value_mh,
    "Low vs High Z-Statistic": z_stat_lh,
    "Low vs High P-Value": p_value_lh,
}

results

**Low Discounts (0-30%) vs Medium Discounts (30-60%)**:
- p-value is less than 0.05 (0.000813), so we reject the null hypothesis.
- There is a significant difference in the proportion of highly rated products between low and medium discounts.
- Z-statistic = 3.35, which means low discounts rate is 3 times of standard deviation above the mean. So products with low discounts (0-30%) have higher proportion of high ratings compared to medium discounts (30-60%)

**Medium Discounts (30-60%) vs High Discounts (60-100%)**:
- p-value is greater than 0.05 (0.301), so we don't have evident to reject the null hypothesis.
- There is no statistically significant difference in the proportion of highly rated products between medium and high discounts.
- Products with medium discounts (30–60%) and high discounts (60–100%) have similar proportions of high ratings.

**Low Discounts (0-30%) vs High Discounts (60-100%)**:
- p-value is less than 0.05 (0.00007), so we reject the null hypothesis.
- There is a significant difference in the proportion of highly rated products between low and high discounts.
- Z-statistic = 3.967, which means low discounts rate is 4 times of standard deviation above the mean. So products with low discounts (0-30%) have higher proportion of high ratings compared to high discounts (60-100%)  

**Summary**:
<center>

|Comparison|Statistical Significance|Conclusion|
| :- | :- | :- |
|**Low vs Medium**|Yes (p = 0.0008)|Low discounts have a higher proportion of high ratings.|
|**Medium vs High**|No (p = 0.3012)|No significant difference in high ratings.|
|**Low vs High**|Yes (p = 0.00007)|Low discounts have a higher proportion of high ratings.|

</center>

<a id='question2'></a>
### Research Question 2:  Do products with more reviews have higher or lower average ratings?

**Analysis Decision 1**: Create review count ranges.

In [57]:
df_clean['rating_count'].describe()

- The review counts range is quite wide, from 2 to 426,973 reviews.
- The mean is 18283, std is 42741 shows that the distribution is skewed.

**Analysis Decision 2**: Visualize the distribution of review bin group.

In [58]:
plt.figure(figsize=(14, 8))
sns.histplot(df_clean['rating_count_group'], kde=True)
plt.title('Distribution of Review bin group')
plt.xlabel('Reivews')
plt.ylabel('Frequency')
plt.show()

**Summary**:
- The rating_count_group binning has provided a more balanced view of data.
- The bimodal distribution suggests that most products either have moderate (1000-5000) or high (10000-50000) review counts, with fewer products at the extremes (very low or very high review counts). This information can help guide further analyses on how review count affects product ratings, with attention to the most common review groups.

**Analysis Decision 3**: Calculate Average Ratings for Each Review Count Group

In [59]:
# Group by the review count ranges and calculate the mean rating
review_count_vs_rating = df_clean.groupby('rating_count_group')['rating'].mean().reset_index()

In [60]:
# Display the result
print(review_count_vs_rating)

**Reasoning**: 

**Summary**:
- The average rating generally increases with the review count, suggesting that products with more reviews tend to have slightly higher ratings. However, this increase plateaus after the 50000-100000 review group, with a slight decline in the 100000+ group.

**Analysis Decision 4**: Visualize the Relationship

In [61]:
# Plot the relationship between review count groups and average ratings
plt.figure(figsize=(14, 8))
plt.bar(review_count_vs_rating['rating_count_group'], review_count_vs_rating['rating'], color='skyblue')
plt.xlabel('Review Count Groups')
plt.ylabel('Average Rating')
plt.title('Average Rating by Review Count Groups')
plt.xticks(rotation=45)
plt.show()

The bar plot of average ratings by review count groups confirms this trend, with a gradual rise in average rating as review count increases, peaking at 50000-100000 reviews.

In [62]:
plt.figure(figsize=(14,8))
sns.boxplot(data=df_clean, x='rating_count_group', y='rating')
plt.title('Box Plot of Ratings Across Review Count Bins')
plt.xlabel('Number of Reviews (Binned)')
plt.ylabel('Rating')
plt.show()

**Summary**: 
- Products with fewer reviews have more variability in their ratings (the spread is wider then higher review counts).
- Products with moderate to higher reivews are more consistent and positive in their ratings, clustering around 4.0.
- Products have 100.000+ reviews slide downward.
- All data set reamin mostly positive (above 3.5)

**Analysis Decision 5**: Correlation Analysis

In [63]:
correlation = df_clean['rating_count'].corr(df_clean['rating'])
print(f"Correlation between Number of Reviews and rating: {correlation}")

In [64]:
sns.scatterplot(x='rating',y='rating_count', data= df_clean, color = "green")
plt.title("Relationship Between Number of Reviews and Ratings")
plt.xlabel('Rating')
plt.ylabel("Number of Reviews")
plt.show()

- The correlation value of 0.102 suggests that there is a slight positive relationship between the number of reviews and its rating, but this is very weak.
- products with more reviews tend to have slightly higher ratings but the effect is minimal.

In [65]:
# Group by rating categories and calculate the correlation
grouped_by_rating = df_clean.groupby('mean_of_ratings').apply(
    lambda x: x['rating_count'].corr(x['rating'])
)

print(grouped_by_rating)

- Low-rated product has a moderate positive correlation (0.604), suggesting that among low-rated products, those with more reviews may have slightly higher ratings.
- Average-rated product has weak positive correlation (0.211).
- Good-rated product has very weak positive correlation (0.136).
- Excellent-rated product has weak negative correlation (-0.065), indicating that for top-rated products, an increase in review count does not necessarily correlate with higher ratings.

In [66]:
review_count_vs_rating = df_clean.groupby('mean_of_ratings')['rating_count'].mean()
print(review_count_vs_rating)

- Average review count of low-rated products is only 199, indicating that lower-rated products are generally reviewed less.
- Average review count of excellent-rated products is significantly higher at 22,619, indicating that highly-rated products tend to have more reviews, potentially reflecting greater popularity or satisfaction.

**Analysis Decision 6**: Build an SVM model

**Reasoning** : 
- The dataset mostly has Excellent Ratings (4.0 - 4.5) and Exellent Ratings (4.5 - 5.0) so we cannot use Classification model.
- The correlation between Numbers of Review and Rating is 0.102, indicating a very weak linear relationship. This suggests that the relationship might be non-linear.

In [67]:
# Log-transform the number of reviews to reduce skewness 
df_clean['log_number_of_reviews'] = np.log1p(df_clean['rating_count'])

In [68]:
# Features and target
X = df_clean[['log_number_of_reviews']]  # Number of reviews
y = df_clean['rating']

In [69]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
# Standardize the feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [71]:
# Initialize and train the SVR model
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.2)  # Adjust C and epsilon for better performance
svr_model.fit(X_train_scaled, y_train)

In [72]:
# Make predictions
y_pred = svr_model.predict(X_test_scaled)

In [73]:
# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [74]:
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

**Mean Squared Error**:
- The MSE represents the average squared difference between the predicted and actual ratings.
- The MSE is quite small (0.067), suggesting a accurated predict

**Mean Absolute Error**:
- The MAE shows the average absolute error between predicted and actual ratings.
- On average, the model's predictions are off by approximately 0.19 points

**R-squared (R²)**:
- The R-squared value indicates that the model explains about 5.18% of the variance in ratings based on the predictors.
- While positive, this is still a very small proportion, suggesting there is room for improvement.

**Summary**:
- Very Weak Relationship: The low R-squared indicates that the number of reviews alone does not strongly predict ratings. This weak relationship aligns with the low correlation coefficient (0.102).
- This suggests that the number of reviews is not a major determinant of ratings. Additional features are likely needed to improve the model.

<a id='question3'></a>
### Research Question 3:  Do higher-priced products receive better ratings than lower-priced ones?

**Analysis Decision 1**: Create review count ranges.

In [75]:
plt.figure(figsize=(14, 8))
sns.histplot(df_clean['discounted_price'], bins=30, kde=True)
plt.title('Distribution of Rating')
plt.xlabel('Discounted Price')
plt.ylabel('Frequency')
plt.show()

**Analysis Decision 2**: Create review count ranges.

In [76]:
# Group by price category and calculate the mean rating
price_rating_comparison = df_clean.groupby('price_category')['rating'].mean()
print(price_rating_comparison)

- Higher-priced products (4.11) have a slightly higher average rating than lower-priced products(4.08).
- The difference is small (0.03), suggesting that price does not strongly influence rating.

In [77]:
df_clean.groupby('price_category')['rating'].describe()

- Both prices have a similar spread of ratings
- Higher-Priced products showing slightly less variation (lower standard deviation) than Lower-Priced products.
- This suggests that higher-priced products tend to have more consistent ratings, though the difference is minor.

In [78]:
# Create a boxplot for ratings by price category
plt.figure(figsize=(14, 8))
sns.boxplot(x='price_category', y='rating', data=df_clean)
plt.title('Product Ratings by Price Category')
plt.xlabel('Price Category')
plt.ylabel('Product Rating')
plt.show()

- Higher-priced products tend to receive consistently good ratings with fewer very low ratings compared to lower-priced products, which exhibit more variability in ratings and have more instances of low ratings.
- Median Rating remains the same in both price categories. It shows that customers generally rate products well regardless of price.

In [79]:
# Group by rating categories and calculate the correlation
grouped_by_rating = df_clean.groupby('price_category').apply(
    lambda x: x['actual_price'].corr(x['rating'])
)

print(grouped_by_rating)

- Correlation between higher-priced products rating is 0.161, indicating a weak positive relationship between price and rating in this category.
- Correlation between lower-priced products rating is 0.048, showing an even weaker positive relationship.
- This suggests that within each group (Higher-Priced and Lower-Priced), products with higher prices tend to receive slightly better ratings, but the relationship is not strong.

**Analysis Decision 3**: Hypothesis Testing.

- Null Hypothesis ($H_0$): There is no significant difference in ratings between lower-priced and higher-priced products.
- Alternative Hypothesis ($H_1$): Higher-priced products receive significantly different ratings.

In [80]:
# Create lower-priced and higher-priced groups
lower_priced = df_clean[df_clean['price_category'] == 'Lower-Priced']
higher_priced = df_clean[df_clean['price_category'] == 'Higher-Priced']

In [81]:
# Perform t-test
t_stat, p_value = ttest_ind(lower_priced['rating'], higher_priced['rating'])

print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_value}")

- t_statistic is negative (-2.098), it shows that the mean rating of lower-priced products is lower than the mean rating of higher-priced products.
- p-value is less than 0.05, we reject the null hypothesis, there is statistically significant difference in ratings between lower-priced and higher-price products.

**Summary**:
- Higher-priced products tend to receive better ratings than lower-priced products, based on this analysis.
- The difference is statistically significant at the 5% level, meaning there’s less than a 5% probability that this result is due to random chance.

**Analysis Decision 3**: Model the Relationship.

In [82]:
from sklearn.linear_model import LinearRegression

X = df_clean[['actual_price']]
y = df_clean['rating']

model = LinearRegression()
model.fit(X, y)

print(f"Regression Coefficient (Price): {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

The price coefficient is very small (0.000005), so the effect of price on ratings is negligible.

In [83]:
r2 = r2_score(y_test, y_pred)

In [84]:
print(f"R-squared (R²): {r2}")

R-squared is very low, it shows that price alone is not a strong predictor of ratings.

<a id='question4'></a>
### Research Question 4:  Do Products with Positive Sentiment in Reviews Have Higher Ratings ?<a href="#top"></a>

**Analysis Decision 1**: Group by Sentiment and Calculate the Average Rating

In [85]:
# Group by sentiment and calculate the average rating
sentiment_rating = df_clean.groupby('vader_sentiment')['rating'].mean().reset_index()
print(sentiment_rating)

- Products with positive sentiment in reviews have a higher average rating compared to those with neutral or negative sentiment. 
- The difference between negative and positive sentiment ratings is about 0.23 points, which indicates a noticeable increase in rating for positively perceived products.

In [86]:
# Sentiment distribution
sentiment_counts = df_clean['vader_sentiment'].value_counts()
plt.figure(figsize=(14, 8))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette='viridis')
plt.title('Distribution of Review Sentiments')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

# Summary
# - Most reviews express positive sentiment, aligning with the generally high ratings.
# - Negative sentiment reviews are relatively low, reflecting lower instances of poor product experiences.

### Summary ###
- Most reviews express positive sentiment, aligning with the generally high ratings.
- Negative sentiment reviews are relatively low, reflecting lower instances of poor product experiences.

In [87]:
# Sentiment distribution by price category
plt.figure(figsize=(14, 8))
sns.countplot(x='price_category', hue='vader_sentiment', data=df_clean, palette='coolwarm')
plt.title('Sentiment Distribution by Price Category')
plt.xlabel('Price Category')
plt.ylabel('Count')
plt.show()

In [88]:
# Compare average discount and rating across categories
category_comparison = df_clean.groupby('main_category').agg({'discount_percentage': 'mean', 'rating': 'mean'}).reset_index()
# Sort by discount_percentage in descending order
category_comparison = category_comparison.sort_values(by='discount_percentage')
plt.figure(figsize=(14, 8))
sns.barplot(x='main_category', y='discount_percentage', data=category_comparison, color='lightblue', label='Average Discount %')
sns.barplot(x='main_category', y='rating', data=category_comparison, alpha=0.5, color='darkblue', label='Average Rating')
plt.xticks(rotation=45)
plt.title('Average Discount Percentage and Rating by Category')
plt.xlabel('Category')
plt.legend()
plt.show()

In [89]:
# Bar plot for average rating by sentiment
plt.figure(figsize=(14, 8))
sns.barplot(x='vader_sentiment', y='rating', data=df_clean, estimator=np.mean)
plt.title('Average Rating by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Average Rating')
plt.show()

As the bar graph shows:
- Products receive more positive reviews will also receive higher ratings.
- Both neutral and negative sentiment products have fairly close average ratings, though slightly lower than positive ones.
- This shows that neutral viewers aren't significantly driving up ratings, and negative feedback doesn't drastically reduce their rating either.

In [90]:
df_clean.groupby('vader_sentiment')['rating'].describe()

- Positive sentiment reviews are associated with higher average ratings and slightly lower variability, suggesting that positive sentiment aligns with more consistent high ratings.
- The standard deviation of Neutral and negative are quite big (0.4 and 0.6 compared with 0.3), it shows that customers who have neutral and negative reviews tend to have wider range in rating.

In [91]:
# Box plot to show the distribution of ratings by sentiment
plt.figure(figsize=(14, 8))
sns.boxplot(x='vader_sentiment', y='rating', data=df_clean)
plt.title('Distribution of Ratings by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Rating')
plt.show()

- Products with positive sentiment tend to have higher ratings, but there are still some lower-rated products with positive sentiment reviews.
- Products with negative sentiment have a wider distribution of ratings, including some very low ratings, though the median is still relatively high.
- The neutral sentiment group has a consistent range of ratings, without extreme highs or lows.

In [92]:
correlation = df_clean['vader_sentiment_score'].corr(df_clean['rating'])
print(f"Correlation between sentiment score and rating: {correlation}")

**Reasoning**:

**Summary**: 
- Since the correlation is positive, it means that as the sentiment score increases (i.e., reviews become more positive), the product rating tends to increase as well. However, the correlation is relatively weak.
- A correlation of 0.238 suggests that while there is a relationship between sentiment and rating, it is not very strong. This means other factors may also be influencing the product ratings apart from sentiment. It's possible that even with positive sentiment, product quality, expectations, or other factors might moderate the final rating a customer leaves.

**Analysis decision **: Statistical Test: ANOVA

In [93]:
positive = df_clean[df_clean['vader_sentiment'] == 'Positive']['rating']
negative = df_clean[df_clean['vader_sentiment'] == 'Negative']['rating']
neutral = df_clean[df_clean['vader_sentiment'] == 'Neutral']['rating']

# Perform one-way ANOVA
anova_result = stats.f_oneway(positive, negative, neutral)
print(f"ANOVA result: {anova_result}")

1. F-statistic (24.996):

- The F-statistic measures the variance between the groups (Positive, Negative, Neutral sentiments) compared to the variance within the groups.
- A higher F-statistic suggests that the variance between groups is significantly larger than the variance within groups, indicating that the differences in the groups are meaningful.
2. p-value (2.12e-11):

- The p-value is extremely small, much lower than the commonly used significance threshold of 0.05.
- This very low p-value indicates that the differences in ratings across the sentiment categories (Positive, Negative, and Neutral) are statistically significant.
- Therefore, we reject the null hypothesis, which states that there is no difference in ratings based on sentiment. There is a significant difference in product ratings based on the sentiment expressed in the reviews.

<a id='question5'></a>
### Research Question 5: Which Categories of Products Receive the Highest or Lowest Ratings ?

In [94]:
df_clean['main_category'].value_counts()

As we can see, `MusicalInstruments`, `HomeImprovement`, `Toys&Games`, `Car&Motorbike`, `Health&PersonalCare`, `OfficeProducts` have only 1 or 2 product, so we cannot analysis based on small sample size.

In [95]:
# Count the number of products in each category
category_counts = df_clean.groupby('main_category').size().reset_index(name='product_count')

# Filter to get only categories with 3 or more products
valid_categories = category_counts[category_counts['product_count'] >= 100]['main_category']

# Filter the original DataFrame to include only these valid categories
filtered_df = df_clean[df_clean['main_category'].isin(valid_categories)]

# Display the filtered DataFrame
print(filtered_df['main_category'].value_counts())

Group by category_1

In [96]:
# Group by category_1 and calculate the mean rating
category_rating = filtered_df.groupby('main_category')['rating'].mean().reset_index()

# Sort by rating to see highest and lowest
category_rating = category_rating.sort_values(by='rating', ascending=False)
print(category_rating)

- All categories have relatively high ratings (above 4.0 on average)
- Computers & Accessories stands out with the highest ratings, indicating a slightly better customer perception in this category.

In [97]:
# Bar plot of average ratings by product category
plt.figure(figsize=(14, 8))
sns.barplot(x='rating', y='main_category', data=filtered_df, hue='main_category', palette='viridis', dodge=False, legend=False)
plt.title('Average Rating by Product Category')
plt.xlabel('Average Rating')
plt.ylabel('Product Category')
plt.show()

- All categories have average ratings above 4.0, suggesting a good level of customer satisfaction across the board.
- Computers & Accessories leads slightly in customer satisfaction, possibly reflecting better perceived quality or value in this category compared to Electronics and Home & Kitchen.

In [98]:
# Group by rating categories and calculate the correlation
grouped_by_rating_cat = filtered_df.groupby(['main_category','mean_of_ratings']).apply(
    lambda x: x['discount_percentage'].corr(x['rating']) if len(x) > 1 else None
)

print(grouped_by_rating_cat)

**Summary**: 
- Electronics with low ratings show the strongest negative correlation (-0.842989), it shows that higher discounts are more common for lower-rated electronics.
- Home & Kitchen Low-rated products (0.594894) and Computers & Accessories good-rated products (0.164417) show positive correlations, suggesting that discounts may be associated with better ratings in certain contexts within these categories.
- Across all categories, the correlation between discounts and ratings is minimal for "Excellent" rated products, implying that discounts don’t significantly impact ratings for highly-rated items

# <a id='conclusions'></a>
## Conclusions


1. Is there a relationship between discount percentage and average product rating ?
- The relationship is weak negative between discount percentage and average product rating.
- Lower-rated products often have higher discount, while higher-rate products seem not have significantly impact.
2. Do products with more reviews have higher or lower average ratings?
- There is a weak positive relationship between the number of reviews and average product ratings. Products with more reviews tend to have slightly higher ratings, but the effect is small.
- Products with more reviews tend to have tighter rating distributions, suggesting that increased visibility may lead to more consistent ratings, potentially due to a larger and more diverse customer base.
- Higher review counts have a stronger positive impact on lower-rated products, potentially helping improve their ratings slightly. However, for "Excellent" rated products, the effect of additional reviews on rating is minimal or even slightly negative.
3. Do higher-priced products receive better ratings than lower-priced ones?
- Higher-priced products receive better ratings than lower-priced products, but the difference is minimal, indicating that price alone is not a major determinant of product rating.
- Higher-priced products show slightly more consistent ratings (lower standard deviation), suggesting that customers may have more stable expectations for higher-priced items.
- The weak positive correlation between price and rating within both groups implies that customers slightly favor higher-priced products, but this effect is minor and likely influenced by other factors such as product quality or brand reputation.
4. Do Products with Positive Sentiment in Reviews Have Higher Ratings?
- Products with positive sentiment in reviews tend to have higher ratings compared to those with neutral or negative sentiment, confirming that sentiment is a useful indicator of product rating.
- The ANOVA test confirms that these differences are statistically significant, suggesting that the sentiment expressed in reviews is a meaningful predictor of product rating.
- For businesses, encouraging positive customer sentiment in reviews can positively impact product ratings. This highlights the importance of customer satisfaction and quality in maintaining high ratings.
5. Which Categories of Products Receive the Highest or Lowest Ratings?
- Computers & Accessories has the highest average rating (4.16), suggesting that this category generally has a higher level of customer satisfaction.
- Home & Kitchen has the lowest average rating among the analyzed categories, though the difference is minimal, indicating that all three categories generally receive favorable ratings.
- In Electronics and Computers & Accessories, lower-rated products tend to have higher discounts, which may indicate a strategy to boost sales or attractiveness despite lower customer satisfaction.
- Home & Kitchen shows a unique pattern where low-rated products actually benefit in ratings with higher discounts, as indicated by the positive correlation for Low ratings.

<a id='limit'></a>
## Limitation

- This source might not represent a comprehensive and unbiased sample of all movies. It is possible that some genres of movies are not represented.
- The correlation does not imply causation. It will need more experiments and statistical analyses. The findings in this analysis provide associations but should not be interpreted as causal relationships.

In [99]:
from subprocess import run
result = run(['python', '-m', 'nbconvert', '--to', 'html', 'Investigate_a_Dataset.ipynb'])
print("Conversion successful!" if result.returncode == 0 else "Conversion failed.")

<a href="#top" style="position: fixed; bottom: 10px; right: 10px; background-color: #28a745; color: white; padding: 8px 12px; border-radius: 5px; text-decoration: none; font-family: Arial;">Return to Top</a>